In [1]:
#In this notebook, we perform and analyze all GST resampling
#in order to be able to compute RMS errors.
#The analogous task for RPE resampling
#is performed in RMS_Error_RPE.ipynb.

#Note that this notebook takes about 4 hours
#to run on a modern laptop.  Some users may
#find it easier to run as a python script instead.
#This notebook may be exported as a .py file:
#File --> Download as --> Python (.py).

In [2]:
#Import relevant namespaces.
import pygsti as gst
import numpy as np
import pygsti.construction.std1Q_XY as Std1Q_XY
import time
import pickle
from pygsti.tools import rpe as RPE
from pygsti.construction import rpeconstruction as RPEConstr
from pygsti.tools.rpeconfig_GxPi2_GyPi2_UpDn import rpeconfig_GxPi2_GyPi2_UpDn

Fully specified RPE configuration.


In [3]:
#Declare the particular RPE instance we are interested in
#(X and Y pi/2 rotations)
rpeconfig_inst = rpeconfig_GxPi2_GyPi2_UpDn

In [4]:
#Declare a variety of relevant parameters

gs_target = Std1Q_XY.gs_target
gs_target.set_all_parameterizations('TP')
maxLengths_1024 = [0,1,2,4,8,16,32,64,128,256,512,1024]
fiducials = Std1Q_XY.fiducials
#measFiducials = Std1Q_XY.fiducials
germs = gst.construction.gatestring_list([('Gy',),
 ('Gy','Gy','Gy','Gx',),
 ('Gy','Gx','Gy','Gx','Gx','Gx',),
 ('Gy','Gx','Gy','Gy','Gx','Gx',),
 ('Gy','Gy','Gy','Gx','Gy','Gx',),
 ('Gx',),
 ('Gx','Gy',),
 ('Gx','Gx','Gy','Gx','Gy','Gy',)])
stringListsGST = gst.construction.make_lsgst_lists(gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths_1024)

stringListsRPE = RPEConstr.make_rpe_angle_string_list_dict(10,rpeconfig_inst)

angleList = ['alpha','epsilon','theta']

NList = [8,16,32,64,128,256]

numKs = len(maxLengths_1024[1:])
numKs_w_0 = len(maxLengths_1024)

numStrsD = {}
numStrsD['RPE'] = [6*i for i in np.arange(1,12)]
numStrsD['GST'] = [len(stringList) for stringList in stringListsGST][1:]

In [5]:
#Load the experimental datasets.
DSGST = gst.io.load_dataset('GST_dataset.txt',cache=True)
DSRPE = gst.io.load_dataset('RPE_dataset.txt',cache=True)

Loading RPE_dataset.txt: 100%
Writing cache file (to speed future loads): RPE_dataset.txt.cache


In [6]:
#Number of times we want to resample to compute RMS error.
trials = 100

In [7]:
#Create empty dictionaries and arrays to hold various results.

gsEstD = {}
resultsD = {}
trueD = {}

method = 'GST'
for angle in angleList:
    resultsArray = np.zeros([numKs,trials,3],float)
    resultsD[(angle,method,'Exp. N=370')] = resultsArray.copy()

for method in ['GST','RPE']:
    for N in NList:
        expKey = ('method',method,'N',N)
        for angle in angleList:
            resultsArray = np.zeros([numKs,trials,3],float)
            resultsD[(angle,)+expKey] = resultsArray.copy()

In [8]:
#Define auxiliary function to resample from a dataset without replacement.

def resample_dataset(input_dataset,N,seed=None):
    if seed is not None:
        np.random.seed(seed)
    keys = input_dataset.keys()
    output_dataset = gst.objects.dataset.DataSet(spamLabels=input_dataset.get_spam_labels())
    for gate_string in keys:
        temp_array = [1]*input_dataset[gate_string]['plus'] + [0]*input_dataset[gate_string]['minus']
        np.random.shuffle(temp_array)
        n = np.sum(temp_array[:N])
        output_dataset.add_counts_1q(gate_string,n,N-n)
    return output_dataset

In [9]:
#Create and analyze 100 resampled datasets for several values of N.
#This should run in less than a minute.
seedCounter = 0
method = 'GST'
startAll = time.time()
for N in NList:
    expKey = ('method',method,'N',N)
    startN = time.time()
    for index1 in xrange(trials):
        print "N, index1 =", N, index1
        DSTemp = resample_dataset(DSGST,N,seed=seedCounter)
        resultsD[('seedCounter-index1',index1)+expKey] = seedCounter
        seedCounter += 1
        GSTresultsTemp = gst.do_long_sequence_gst(DSTemp,gs_target,fiducials,fiducials,germs,maxLengths_1024,verbosity=0, gaugeOptParams = {'itemWeights': {'gates':1.0, 'spam':1e-4}})
        for kInd, k in enumerate(maxLengths_1024[1:]):
            gs = GSTresultsTemp.gatesets['iteration estimates'][kInd]#gst.optimize_gauge(gsEstD[baseKey][kInd],'target',targetGateset=gs_target,constrainToTP=True,spamWeight=1e-4)
            resultsD[('alpha',)+expKey][kInd,index1,0] = RPE.extract_alpha(gs,rpeconfig_inst)
            resultsD[('epsilon',)+expKey][kInd,index1,0] = RPE.extract_epsilon(gs,rpeconfig_inst)
            resultsD[('theta',)+expKey][kInd,index1,0] =  RPE.extract_theta(gs,rpeconfig_inst)                
        del GSTresultsTemp
    endN = time.time()
    print endN - startN
endAll = time.time()
print endAll - startAll

N, index1 = 8 0
DEBUG: optimized gate to min frobenius distance 0.0275849
DEBUG: optimized gate to min frobenius distance 0.0262204
DEBUG: optimized vector to min frobenius distance 0.000650511


/Users/kmrudin/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


KeyboardInterrupt: 

In [17]:
#Save above results, stored in the dictionary resultsD, as a pickle.
GSTResultsFile = open('GST-Subsampling-Results-new.pkl','w')
pickle.dump(resultsD,GSTResultsFile)
GSTResultsFile.close()